# **Recommendation Agent**

The **Recommendation_Agent** is the final agent in the recommendation process. It orchestrates how the **Activity_Agent**, **Usage_Agent**, **Load_Agent** and **Price_Agent** interact in order to deliver a recommendation at a particular date for a given household (see the dedicated notebooks in order to see the definition of these agents in detail).

The recommendation agent works as follows:

1. It requests the outputs of the **Activity_Agent**, **Usage_Agent**, **Load_Agent** and **Price_Agent**:


*   The activity agent returns the probability that persons are present and in an "active state" in the house at each given hour of the day.
*   The usage agent returns the probability that a given to-be-recommended-device will be used on the next day.
*   The load agent returns a typical load profile for each to-be-recommended-device
*   The price agent returns the day-ahead-prices for the next 48 hours

2. It then computes the cost associated with launching the devices at each hour of he next day (based on the devices' typical load profiles and the day ahead electricity prices).

3. Finally it recommends the cheapest launching hour among the set of hours at which users are likely present and active **[Probability(Present&Active)> Threshold]**. A recommendation  for a given device is made only if the user is likely enough to use the device on the next day **[Probability(Device_Usage)> Threshold]**


In the present script, we will build this **Recommendation_Agent** step by step. For that purpose, we will first load and preprocess the required data with the preprocessing agents. Then, we will iteratively add functions to the **Recommendation_Agent class** in order to finally build a function entitled "Pipeline", which ouputs the desired recommendations.

## **1. Load and Pre-process Data**

This part's only purpose is to load the data used in the recommendation agent. This process is described in detail in the Preparation_Agent.  **[You might need to adapt some parameters when applying the script to another household than household 1]**

### **1.1 Initialize and load python scripts**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = '/content/drive/MyDrive/T4_Recommendation-system-for-demand-response-and-load-shifting/02_data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading .py scripts of prior agents and helper functions

# copy scripts to colab
!cp /content/drive/MyDrive/T4_Recommendation-system-for-demand-response-and-load-shifting/03_scripts/helper_functions.py .
!cp /content/drive/MyDrive/T4_Recommendation-system-for-demand-response-and-load-shifting/03_scripts/agents.py .

In [ ]:
# loading necessary libraries
import pandas as pd
import numpy as np

from helper_functions import Helper
from agents import Preparation_Agent

helper = Helper()

### **1.2 Set Params**

Note: For the full detail of the parameters and the preprocessing agents take a look at the Preparation_Agents notebook.

In [ ]:
# load household data for Household 1
household = helper.load_household(DATA_PATH, 1)

In [ ]:
#global params
threshold = .15
active_appliances = ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site']
shiftable_devices = ['Tumble Dryer', 'Washing Machine', 'Dishwasher']

#activity params

truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}

activity_params = {
    'active_appliances': active_appliances,
    'threshold': threshold 
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

#load agent

device_params = {
    'threshold': threshold
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

#usage agent

device = {
    'threshold' : threshold}

aggregate_params24_H = {
    'resample_param': '24H'
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}


In [ ]:
# calling the preparation pipeline
prep = Preparation_Agent(household)
activity_df = prep.pipeline_activity(household, activity_pipe_params)
load_df, _, _ = prep.pipeline_load(household, load_pipe_params)
usage_df = prep.pipeline_usage(household, usage_pipe_params)

#load price data
FILE_PATH = '/content/drive/MyDrive/T4_Recommendation-system-for-demand-response-and-load-shifting/02_data/' 
price_df = helper.create_day_ahead_prices_df(FILE_PATH, 'Day-ahead Prices_201501010000-201601010000.csv')

## **2. Constructing the Recommendation Agent**

### **2.1 Initiliaze Agent**
In a first step, the recommendation agent is initialized with the preprocessed data and the name of the shiftable_devices (those for which we want to make predictions). All Agents are initialized accordingly.

In [ ]:
from agents import Activity_Agent, Usage_Agent, Load_Agent, Price_Agent
class Recommendation_Agent:
    import pandas as pd

    def __init__(self, activity_input, usage_input, load_input, price_input, shiftable_devices):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices

        self.Activity_Agent = Activity_Agent(activity_input)

        #create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {name: Usage_Agent(usage_input , name)  for name in shiftable_devices}

        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)

In [ ]:
#initialize the Recommendation Agent with the required inputs
recommend = Recommendation_Agent(activity_df, usage_df, load_df, price_df, shiftable_devices)

### **2.2 Compute Usage Cost For Every Starting Time (each of the 24 hours of the day) For A Given Device**

This function computes the cost associated with launching a to-be-recommended-device at each hour of the next day. 

#### **2.2.1 Electricity Prices For 24 hours After Hypothetical Starting Time**
First we build up a function which gives the electricity price for the 24 hours following a hypothetical starting time. 

*   The column "Price_at_H+0" gives the electricity price for the 24 hours after 00:00:00
*   The column "Price_at_H+1" gives the electricity price for the 24 hours following 01:00:00 ( "Price_at_H+0" shifted by one hour)
* The column "Price_at_H+2" gives the electricity price for the 24 hours following 02:00:00 ( "Price_at_H+0" shifted by two hours)
*....

This function first requests the **day-ahead electricity prices** for the next 48h from the Price_Agent. Then it arranges the prices as described above.


In [ ]:
def electricity_prices_from_start_time(self, date):
  import pandas as pd
  prices_48 = self.Price_Agent.return_day_ahead_prices(date)
  prices_from_start_time = pd.DataFrame()

  for i in range(24):
    prices_from_start_time["Price_at_H+"+ str(i)] = prices_48.shift(-i)

  #delete last 24 hours
  prices_from_start_time = prices_from_start_time[:-24]
  return prices_from_start_time

# add to Activity agent
setattr(Recommendation_Agent, 'electricity_prices_from_start_time', electricity_prices_from_start_time)
del electricity_prices_from_start_time

In [ ]:
recommend.electricity_prices_from_start_time("2014-02-20")
#H0 prices for next 24 hours start at 00:00
#H1 prices for next 24 hours start at 01:00

,Price_at_H+0,Price_at_H+1,Price_at_H+2,Price_at_H+3,Price_at_H+4,Price_at_H+5,Price_at_H+6,Price_at_H+7,Price_at_H+8,Price_at_H+9,Price_at_H+10,Price_at_H+11,Price_at_H+12,Price_at_H+13,Price_at_H+14,Price_at_H+15,Price_at_H+16,Price_at_H+17,Price_at_H+18,Price_at_H+19,Price_at_H+20,Price_at_H+21,Price_at_H+22,Price_at_H+23
2014-02-20 00:00:00,33.95,33.96,33.52,31.58,31.24,35.55,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06
2014-02-20 01:00:00,33.96,33.52,31.58,31.24,35.55,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93
2014-02-20 02:00:00,33.52,31.58,31.24,35.55,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28
2014-02-20 03:00:00,31.58,31.24,35.55,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41
2014-02-20 04:00:00,31.24,35.55,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41,32.96
2014-02-20 05:00:00,35.55,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41,32.96,30.23
2014-02-20 06:00:00,42.94,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41,32.96,30.23,30.75
2014-02-20 07:00:00,41.23,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41,32.96,30.23,30.75,34.15
2014-02-20 08:00:00,43.59,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41,32.96,30.23,30.75,34.15,34.02
2014-02-20 09:00:00,55.98,51.90,51.60,44.01,39.22,37.96,37.73,40.16,55.75,77.92,47.95,42.35,39.57,36.73,38.06,37.93,37.28,34.41,32.96,30.23,30.75,34.15,34.02,38.00


#### **2.2.2 Device Launching Cost By Hour Of The Day**
We compute the cost of operating the device at every given hour by multiplying the **day ahead electricity price** with the device's **typical load_profile**. The latter typical load profile is generated by the **Load_Agent** (for details see the Load_Agent notebook).

As an output, we get the typical costs of operating the device for all possible 24 staring times.

In [ ]:
def cost_by_starting_time(self, date, device):
  import numpy as np
  import pandas as pd
  #get electriciy prices following every device starting hour with previously defined function
  prices = self.electricity_prices_from_start_time(date)

  #build up table with typical load profile repeated for every hour (see Load_Agent)
  device_load = self.Load_Agent.pipeline(self.load_input, date, self.shiftable_devices).loc[device]
  device_load = pd.concat([device_load] * 24, axis= 1)

  #multiply both tables and aggregate costs for each starting hour
  costs = np.array(prices)*np.array(device_load)
  costs = np.sum(costs, axis = 0)

  #return an array of size 24 containing the total cost at each staring hour.
  return costs
  # add to Activity agent

setattr(Recommendation_Agent, 'cost_by_starting_time', cost_by_starting_time)
del cost_by_starting_time

As can be seen below, the output returns the cost associated with starting the "Washing machine" on each hour of the "2014-02-20".

In [ ]:
recommend.cost_by_starting_time("2014-02-20", "Washing Machine")

array([ 5547.64424686,  5538.69827628,  5460.08389051,  5196.34081308,
        5209.57480978,  5931.38772442,  7034.67776022,  6830.15555442,
        7310.23961011,  9074.11886879,  8440.17936408,  8294.58574964,
        7116.26466264,  6399.83810716,  6251.06643316,  6330.53413761,
        6855.66080329,  9326.66585306, 12233.98595725,  7735.16674745,
        6873.15711771,  6433.27937288,  6027.82042456,  6214.7479592 ])

### **2.3 Starting Time Recommendation For Each Device**

We create a function that gives a starting time recommendation for a given device.

1.   The function loads the **device usage costs associated with each starting time** (see above function)
2.   In order **not** to recommend to start the device at an hour where the person is not home or at sleep, we exclude hours that have an  **activity probability below a certain threshold**. These probabilities are computed with the **Activity_Agent**.
3. In order **not** to make a recommendation when the household is unlikely to use the device on the next day anyway, we set a **device usage probability threshold** under which no recommendation is made. These usage probabilities are computed with the **Usage_Agent**.


The function outputs a dictionnary with the best starting time, among the hours at which activity is likely enough. The output additionaly contains **"no_recommend"** flags, in order to signal that no recommendation should be made when :
* There is no hour of the day where activity is likely enough (all hours of the day have an activity probability below the set threshold), then the **"no_recommend_flag_activity"** turns from 0 to 1.
* The device usage is unlikely (below the set threshold), then the **"no_recommend_flag_usage"** turns from 0 to 1.


In [ ]:
#return cheapest launching hour from the set of hours satifsfying:  probability of activity > threshold
def recommend_by_device(self, date, device, activity_prob_threshold, usage_prob_threshold):
  import numpy as np

  #add split params as input
  # IN PARTICULAR --> Specify date to start training
  split_params =  {'train_start': '2013-11-01', 'test_delta': {'days':1, 'seconds':-1}, 'target': 'activity'}

  #compute costs by launching time:
  costs = self.cost_by_starting_time(date, device)

  #compute activity probabilities
  activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, 'logit', split_params)

  #set values above threshold to 1. Values below to Inf 
  #(vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
  activity_probs = np.where(activity_probs>= activity_prob_threshold, 1, float("Inf"))

  #add a flag in case all hours have likelihood smaller than threshold
  no_recommend_flag_activity = 0
  if np.min(activity_probs) == float("Inf"):
    no_recommend_flag_activity = 1

  # compute cheapest hour from likely ones
  best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

  # compute likelihood of usage:
  usage_prob = self.Usage_Agent[device].pipeline(self.usage_input , date, "logit", split_params["train_start"]) 
  no_recommend_flag_usage = 0
  if usage_prob < usage_prob_threshold :
    no_recommend_flag_usage = 1

  return {"recommendation_date": [date], "device": [device] ,"best_launch_hour": [best_hour] , "no_recommend_flag_activity" : [no_recommend_flag_activity], "no_recommend_flag_usage" : [no_recommend_flag_usage] }

setattr(Recommendation_Agent, 'recommend_by_device', recommend_by_device)
del recommend_by_device

In [ ]:
recommend.recommend_by_device("2014-08-21", "Dishwasher", 0.3, 0.3)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


{'best_launch_hour': [3],
 'device': ['Dishwasher'],
 'no_recommend_flag_activity': [0],
 'no_recommend_flag_usage': [0],
 'recommendation_date': ['2014-08-21']}

### **2.4 Create Recommendation Function For Entire Household**
Finally, we wrap up the "recommend_by_device" function that makes a recommendation for each device, into the "pipeline" function that will make recommendations for all shiftable devices within a household.

In [ ]:
def pipeline(self, date, activity_prob_threshold, usage_prob_threshold):
  import pandas as pd
  recommendations_by_device = self.recommend_by_device(date, self.shiftable_devices[0], activity_prob_threshold, usage_prob_threshold)
  recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

  for device in self.shiftable_devices[1:]:
      recommendations_by_device = self.recommend_by_device(date, device, activity_prob_threshold, usage_prob_threshold)
      recommendations_table = recommendations_table.append(pd.DataFrame.from_dict(recommendations_by_device))
  return recommendations_table

setattr(Recommendation_Agent, 'pipeline', pipeline)
del pipeline

We can then generate a recommendation for a household by specifying:
1. The day to be recommended
2. The "activity_prob_threshold" (hours that have a smaller probability of household activity are not considered for recommendation)
3. The "usage_probability_threshold" (devices that have a smaller probability of usage are not considered for recommendation)

**Note**:  It remains to be investigated at which value to set these thresholds.

In [ ]:
recommend.pipeline(date = "2015-02-15",activity_prob_threshold = 0.4,  usage_prob_threshold = 0.3)

,recommendation_date,device,best_launch_hour,no_recommend_flag_activity,no_recommend_flag_usage
0,2015-02-15,Tumble Dryer,8,0,1
0,2015-02-15,Washing Machine,8,0,0
0,2015-02-15,Dishwasher,8,0,1


Finally, the system recommends to the user the "best_launch_hour" on the "recommendation_date" if both the "no_recommend_flag_activity" and "no_recommend_flag_usage" are at 0 for the given device. 

## **Appendix A1: Complete Recommendation Agent**

Here the fully defined class that is copied to the Agent.py file that contains all classes of agents.

In [ ]:
from agents import Activity_Agent, Usage_Agent, Load_Agent, Price_Agent
class Recommendation_Agent:

    def __init__(self, activity_input, usage_input, load_input, price_input, shiftable_devices):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices

        self.Activity_Agent = Activity_Agent(activity_input)

        #create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {name: Usage_Agent(usage_input , name)  for name in shiftable_devices}

        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)

    def electricity_prices_from_start_time(self, date):
        import pandas as pd
        prices_48 = self.Price_Agent.return_day_ahead_prices(date)
        prices_from_start_time = pd.DataFrame()

        for i in range(24):
          prices_from_start_time["Price_at_H+"+ str(i)] = prices_48.shift(-i)

        #delete last 24 hours
        prices_from_start_time = prices_from_start_time[:-24]
        return prices_from_start_time

    def cost_by_starting_time(self, date, device):
        import numpy as np
        import pandas as pd
        #get electriciy prices following every device starting hour with previously defined function
        prices = self.electricity_prices_from_start_time(date)

        #build up table with typical load profile repeated for every hour (see Load_Agent)
        device_load = self.Load_Agent.pipeline(self.load_input, date, self.shiftable_devices).loc[device]
        device_load = pd.concat([device_load] * 24, axis= 1)

        #multiply both tables and aggregate costs for each starting hour
        costs = np.array(prices)*np.array(device_load)
        costs = np.sum(costs, axis = 0)

        #return an array of size 24 containing the total cost at each staring hour.
        return costs

    def recommend_by_device(self, date, device, activity_prob_threshold, usage_prob_threshold):
        import numpy as np
        import pandas as pd

        #add split params as input
        # IN PARTICULAR --> Specify date to start training
        split_params =  {'train_start': '2013-11-01', 'test_delta': {'days':1, 'seconds':-1}, 'target': 'activity'}

        #compute costs by launching time:
        costs = self.cost_by_starting_time(date, device)

        #compute activity
        activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, 'logit', split_params)

        #set values above threshold to 1. Values below to Inf 
        #(vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
        activity_probs = np.where(activity_probs>= activity_prob_threshold, 1, float("Inf"))

        #add a flag in case all hours have likelihood smaller than threshold
        no_recommend_flag_activity = 0
        if np.min(activity_probs) == float("Inf"):
          no_recommend_flag_activity = 1

        # compute cheapest hour from likely ones
        best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

        # compute likelihood of usage:
        usage_prob = self.Usage_Agent[device].pipeline(self.usage_input , date, "logit", split_params["train_start"]) 
        no_recommend_flag_usage = 0
        if usage_prob < usage_prob_threshold :
          no_recommend_flag_usage = 1

        return {"recommendation_date": [date], "device": [device] ,"best_launch_hour": [best_hour] , "no_recommend_flag_activity" : [no_recommend_flag_activity], "no_recommend_flag_usage" : [no_recommend_flag_usage] }

    def pipeline(self, date, activity_prob_threshold, usage_prob_threshold):
      import pandas as pd
      recommendations_by_device = self.recommend_by_device(date, self.shiftable_devices[0], activity_prob_threshold, usage_prob_threshold)
      recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

      for device in self.shiftable_devices[1:]:
        recommendations_by_device = self.recommend_by_device(date, device, activity_prob_threshold, usage_prob_threshold)
        recommendations_table = recommendations_table.append(pd.DataFrame.from_dict(recommendations_by_device))
      return recommendations_table
